In [259]:
# initialise Notebook
import boto3
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import time
import uuid

# key variable initiation
rekognition = boto3.client('rekognition', region_name='us-east-1')
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
s3 = boto3.client('s3')

In [260]:
bucket = "YOUR-S3-BUCKET"
video = "YOUR-Video-File.mp4"

rvddb = dynamodb.Table('YOUR-DynamoDB-TABLE')
pk = bucket + '-' + video

In [261]:
#Label and activity detection
startLabelResponse = rekognition.start_label_detection(
    Video={
        'S3Object': {
            'Bucket': bucket,
            'Name': video,
        }
    },
)

labelJobId = startLabelResponse['JobId']
display(labelJobId)

'7b045835c78dca2d82835d75d02c504777c9fd601dadcbe8e692ef25199b98d6'

In [262]:
#Face detection
startFaceDetectionResponse = rekognition.start_face_detection(
    Video={
        'S3Object': {
            'Bucket': bucket,
            'Name': video,
        }
    },
)

faceDetectionJobId = startFaceDetectionResponse['JobId']
display(faceDetectionJobId)

'b949043b5baaaa45fbd41f38395f805c32a2e9f0eab94dae6afade98f55de91a'

In [263]:
#Person Tracking
startPersonTrackingResponse = rekognition.start_person_tracking(
    Video={
        'S3Object': {
            'Bucket': bucket,
            'Name': video,
        }
    },
)

personTrackingJobId = startPersonTrackingResponse['JobId']
display(personTrackingJobId)

'3cb9cddf48e4405875568f2ae9e69f4f49845994d70fe75922460ed96dc3fb9b'

In [264]:
labelsResponse = rekognition.get_label_detection(
    JobId=labelJobId,
    SortBy='TIMESTAMP'
)

while(labelsResponse['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print(".")
 
    labelsResponse = rekognition.get_label_detection(
    JobId=labelJobId,
    SortBy='TIMESTAMP')
    
display(labelsResponse['JobStatus'])

.
.
.
.
.
.
.
.
.
.
.
.


'SUCCEEDED'

In [265]:
#display(labelsResponse['Labels'][0])

for label in labelsResponse['Labels']:
    response = rvddb.put_item(
                Item={
                      'pk': pk,
                      'sk': str(uuid.uuid4()),
                      'Timestamp': label['Timestamp'],
                      'Activity': str(label['Label']['Name']),
                      'RawJSON' : str(label)
                      }
                )
display('Labels Done!')

'Labels Done!'

In [266]:
#Person Tracking
personTrackingResponse = rekognition.get_person_tracking(
    JobId=personTrackingJobId,
    SortBy='TIMESTAMP'
)

while(personTrackingResponse['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print(".")
 
    personTrackingResponse = rekognition.get_person_tracking(
    JobId=personTrackingJobId,
    SortBy='TIMESTAMP')

display(personTrackingResponse['JobStatus'])

.
.
.
.
.
.


'SUCCEEDED'

In [267]:
#display(personTrackingResponse['Persons'][0])

boundingBox = ''
detectedFace = 0

for person in personTrackingResponse['Persons']:
    if('BoundingBox' in person['Person']):
        detectedFace = 0
        boundingBox = str(person['Person']['BoundingBox'])
    else:
        boundingBox = str(person['Person']['Face']['BoundingBox'])
        detectedFace = 1
    
    response = rvddb.put_item(
                Item={
                      'pk': pk, 
                      'sk': str(uuid.uuid4()),
                      'Timestamp': person['Timestamp'],
                      'Person': person['Person']['Index'],
                      'PersonBoundingBox': boundingBox,
                      'DetectedFace' : detectedFace,
                      'RawJSON': str(person)
                      }
                )
    
display('Person Tracking Done!')

'Person Tracking Done!'

In [268]:
#Face Detection
faceDetectionResponse = rekognition.get_face_detection(
    JobId=faceDetectionJobId
)

while(faceDetectionResponse['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print(".")
        
    faceDetectionResponse = rekognition.get_face_detection(
        JobId=faceDetectionJobId
    )
    
display(faceDetectionResponse['JobStatus'])


'SUCCEEDED'

In [269]:
#display(faceDetectionResponse['Faces'][0])

for face in faceDetectionResponse['Faces']:
    response = rvddb.put_item(
                Item={
                      'pk': pk,
                      'sk': str(uuid.uuid4()),
                      'Timestamp': face['Timestamp'],
                      'Face': str(face['Face']['BoundingBox']),
                      'RawJSON': str(face)
                      }
                )
    
display('Face Detection Done!')

'Face Detection Done!'